# Tutorial para trabalhar com topic modeling usando a biblioteca mllib do spark para aplicações em big data
Este tutorial vai apresentar o conceito de topic modeling superficialmente, o foco dele é como utilizar este método de classificação de texto usando a biblioteca mllib do spark. Essa biblioteca, assim como o spark, vai ser introduzida em mais detalhes conforme necessário para a sua aplicação em contextos genéricos de classificação de textos usando topic modeling.

![alt text](https://www.depends-on-the-definition.com/wp-content/uploads/2018/11/IntroToLDA.png)

## O que é topic modeling e para que serve?
Como introduzido, topic modeling é um método de classificação de textos. Topic modeling define tópicos e calcula a aderência de um texto a cada um destes tópicos, o resultado da aderência a tópico é um coeficiente respectivo. 

Existem várias formas de calcular os coeficientes de aderência de um texto a um conjunto de tópicos, a forma que vai ser usada neste texto é LDA - Latent Dirichlet Allocation. LDA vai ser explicado a seguir no tutorial, ensinar a matemática por trás do algoritmo não é o propósito deste tutorial mas é importante que fique claro como o cálculo LDA é bastante custoso em termos de processamento; ele foi a escolha deste tutorial para classificação de enormes quantidades de texto por que o Spark é uma ferramenta ótima de paralelização de processamento.


---


### Latent Dirichlet... que?
Deixa o nome pra lá por enquanto, ela será chamada de 'a coisa' para que o nome não cause confusão. Vamos focar primeiro no fato de 'a coisa' ser um método de classificação de documentos por tópicos, que tem parâmetros fixos e parâmetros latentes (guarda essa palavra!) - um subconjunto dentre os parâmetros latentes é o conjunto dos tópicos, a quantidade de tópicos é arbitraria. Se pensarmos no algoritmo como algo que fornece a probabilidade de um tópico ser o certo para um dado texto, e simplificando as variaveis um pouco, podemos descrevê-lo da seguinte maneira:

$$\begin{eqnarray}
P (&topico&|&documento,&parametros fixos) = Aderencia&do&texto&ao&topico&
\end{eqnarray}$$

Agora temos um proposito claro, achar o coeficiente de aderencia do texto ao topico. Para chegarmos a esse objetivo, talvez seja mais facil pensar no invérso desta probabilidade, ao invés de tentarmos saber a probabilidade do topico dado o texto nós vamos tentar achar a probabilidade do texto dado o topico; é confuso mas talvez isso ajude: vamos tentar gerar o texto que temos a partir de variar o valor das nossas variaveis que medem aderencia a topico e gerando um bloco de texto com elas.


<h3>Dirichlet</h3>

Como estamos usando LDA (Latent Dirichlet Allocation), é importante entender um pouco como funciona uma Dirichlet, pelo menos para o nosso caso específico.
O que precisamos obter com Dirichlet é a probabilidade de um tópico ser o certo para um dado documento.
Vamos quebrar a explicação para facilitar o entendimento: 

1. escolher o tópico de onde virá esta palavra neste documento

2. escolher a palavra de dentro do tópico escolhido

Coisa do Ayres:

1. escolher o tópico:

  *   para escolher o tópico preciso de: probabilidade de tópico para aquele documento
    *   Cada documento tem suas próprias probabilidades do tópico
        *   "Tópicos do documento"

  
  
  1. 1  Escolher as probabilidades de tópico no documento:

    *   restrições: vetor meio esparso
      *   obter este vetor de probabilidades como uma amostra de uma distribuição de Dirichlet


![alt text](https://miro.medium.com/max/635/1*qwA4jyRFBB6Htn3X4aftSw.png)




2. escolher palavra do tópico:

  *   tenho um vetor de probabilidades de palavra para aquele tópico

  
  
  2. 1  Preciso de probabilidades de palavra por tópico:

    *   restrições: vetor meio esparso
      *   obter este vetor de probabilidades como uma amostra de uma distribuição de Dirichlet




3. Escolhido o tópico, escolher a palavra:



## O que é spark e mllib?
Uma busca rapida no google e você se perde no mar de termos, "You might already know Apache Spark as a fast and general engine for big data processing, with built-in modules for streaming, SQL, machine learning and graph processing", na verdade não, se você ta aqui você não conhece spark e mllib, então vamos nos introduzir a essas duas coisas. Spark é, afinal, uma engine de big data processing... significando que Spark tem seu proprio metodo para distribuir o processamento de uma tarefa para varias maquinas com o intuito de reduzir o tempo que leva para processar big data. A gente precisa de spark, por que as vezes a analize de um problema necessita o processamento de dados, muitos e muitos dados, mais dados do que uma maquina seria capaz de processar. Então, posto que temos tanta informação para processar, vamos utilizar uma aplicação distribuida - spark - que consiga dividir esse bloco de trabalho enorme em bloquinhos menores e fazer maquinas diferentes processarem cada bloquinho para depois obter o mesmo resultado em menos tempo.

![alt text](https://dzone.com/storage/temp/9507196-data-flow2x-768x897.png)

Mllib é a biblioteca de machine learning do spark, ela vai permitir que a gente use LDA de forma otimizada para processamento distribuido que o spark oferece. No tutorial, nós vamos aplicar ela localmente, mas a transição de local para em um cluster na AWS é extremamente facil e vai ser mostrada aqui tambem.

## Para que usar eles ao invés de scikit-learn ou outra biblioteca de machine learning?

Vamos responder esta com uma atividade, no final deste tutorial, onde você vai tentar fazer algo parecido com o que a gente esta prestes a fazer com o spark e mllib, mas com uma biblioteca não distribuida - o scikit-learn; o objetivo desta atividade é ser mostrar que para volumes massivos de informação, é muito mais adequado utilizar uma aplicação distribuida como o spark, uma vez que o tempo para processar essa quantidade de dados em uma maquina se torna inadequado.

## Seria prudente ter uma noção solida dos seguintes conceitos para passar deste ponto:

*   Programação em Python
*   Topic modeling (nada demais, só ter certeza que o conteudo do tutorial até agora ficou claro)

## Para começar, você vai precisar ter acesso a:

*   Uma maquina Ubuntu (O tutorial não foi testado em outros OS's)

#### E nessa maquina, vai ser necessario ter instalado:
*   Python
*   Pip


## Como instalar algumas partes que talvez você não tenha
Supondo que você não tenha já o pyspark, segue como instalar o ele - para usar spark pelo python:

In [1]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 59kB/s 
     |████████████████████████████████| 204kB 51.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=63ff35bf70eb2767a6e3e4f5ccb6c396c5e0640cbec5b8cddd30017b09a3ce64
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


## Começando, vamos iniciar uma seção spark
Essa é a porta de entrada para desenvolvimento em Spark, aqui a gente inicia a seção Spark em cima da qual todo o resto da aplicação vai rodar

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

## Começando, vamos obter dados
Nós vamos usar dados deste db, contendo as letras de musicas:

https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics

(~100MB)

Agora precisamos importar as letras para esse notebook, utilizando o seguinte comando:

In [11]:
from google.colab import files
uploaded = files.upload()


MessageError: ignored

## Dados obtidos, vamos agora 